# AFL Data Cleaning for Prediction Modeling

### Data found on Kaggle [here](https://www.kaggle.com/datasets/stoney71/aflstats)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind, zscore

%matplotlib inline

#Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
games_df = pd.read_csv("games.csv")
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameId         2024 non-null   object 
 1   year           2024 non-null   int64  
 2   round          2024 non-null   object 
 3   date           2024 non-null   object 
 4   venue          2024 non-null   object 
 5   startTime      2024 non-null   object 
 6   attendance     2024 non-null   int64  
 7   homeTeam       2024 non-null   object 
 8   homeTeamScore  2024 non-null   int64  
 9   awayTeam       2024 non-null   object 
 10  awayTeamScore  2024 non-null   int64  
 11  rainfall       1993 non-null   float64
dtypes: float64(1), int64(4), object(7)
memory usage: 189.9+ KB


In [3]:
players_df = pd.read_csv("players.csv")
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   playerId     1495 non-null   int64 
 1   displayName  1495 non-null   object
 2   height       1495 non-null   int64 
 3   weight       1495 non-null   int64 
 4   dob          1495 non-null   object
 5   position     1495 non-null   object
 6   origin       1491 non-null   object
dtypes: int64(3), object(4)
memory usage: 81.9+ KB


In [4]:
stats_df = pd.read_csv("stats.csv")
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89470 entries, 0 to 89469
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gameId                   89470 non-null  object
 1   team                     89470 non-null  object
 2   year                     89470 non-null  int64 
 3   round                    89470 non-null  object
 4   playerId                 89470 non-null  int64 
 5   displayName              89470 non-null  object
 6   gameNumber               89470 non-null  int64 
 7   Disposals                89470 non-null  int64 
 8   Kicks                    89470 non-null  int64 
 9   Marks                    89470 non-null  int64 
 10  Handballs                89470 non-null  int64 
 11  Goals                    89470 non-null  int64 
 12  Behinds                  89470 non-null  int64 
 13  Hit Outs                 89470 non-null  int64 
 14  Tackles                  89470 non-nul

## Merging dataframes to create a dataframe broken down by player and game

In [5]:
df = stats_df.merge(games_df, how='left', on='gameId')\
             .merge(players_df, how='left', on='playerId')
df

,gameId,team,year_x,round_x,playerId,displayName_x,gameNumber,Disposals,Kicks,Marks,Handballs,Goals,Behinds,Hit Outs,Tackles,Rebounds,Inside 50s,Clearances,Clangers,Frees,Frees Against,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs,year_y,round_y,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,displayName_y,height,weight,dob,position,origin
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,2,0,1,0,7,0,2,3,2,2,2,0,4,5,0,0,0,0,1,80,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Berry, Sam",181,81,12-Feb-2002,"Midfield, Forward",Gippsland Power
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,3,0,0,0,2,0,0,0,0,1,0,0,2,3,0,0,0,0,0,23,Off,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Brown, Luke",181,81,22-Sep-1992,Defender,Norwood
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,5,0,0,0,0,5,0,0,3,1,0,0,5,5,1,0,8,0,0,93,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Butts, Jordon",198,83,31-Dec-1999,Defender,Murray Bushrangers
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,4,0,0,0,4,5,1,1,0,2,0,0,8,6,0,0,7,0,0,84,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Doedee, Tom",188,88,1-Mar-1997,Defender,Geelong Falcons
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,4,2,2,7,0,2,0,0,1,0,0,0,5,9,3,4,2,0,0,90,-,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00,"Frampton, Billy",200,90,20-Nov-1996,Forward,South Fremantle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89465,2012R2309,Western Bulldogs,2012,R23,2011838967,"Skinner, Zephaniah",8,2,1,1,1,0,0,0,1,0,1,0,1,0,0,0,2,1,0,0,1,0,0,28,On,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Skinner, Zephaniah",189,79,27-Jun-1989,Forward,-
89466,2012R2309,Western Bulldogs,2012,R23,2012833342,"Smith, Clay",16,22,11,5,11,0,0,0,4,2,3,0,5,2,1,0,6,16,0,0,1,0,0,79,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Smith, Clay",181,85,11-May-1993,Forward,Gippsland Power
89467,2012R2309,Western Bulldogs,2012,R23,2012846405,"Talia, Michael",4,23,14,9,9,0,0,0,1,1,3,0,3,3,2,0,5,18,0,0,3,4,0,88,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Talia, Michael",194,94,11-Feb-1993,Defender,Calder Cannons
89468,2012R2309,Western Bulldogs,2012,R23,2011872415,"Wallis, Mitch",25,19,8,2,11,1,0,0,4,1,3,6,2,3,0,0,9,11,0,0,2,0,0,83,-,2012,R23,02-Sep-2012,Gabba,4:40 PM,18289,Brisbane Lions,128,Western Bulldogs,61,0.00,"Wallis, Mitch",186,85,24-Oct-1992,Forward,Calder Cannons


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89470 entries, 0 to 89469
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gameId                   89470 non-null  object 
 1   team                     89470 non-null  object 
 2   year_x                   89470 non-null  int64  
 3   round_x                  89470 non-null  object 
 4   playerId                 89470 non-null  int64  
 5   displayName_x            89470 non-null  object 
 6   gameNumber               89470 non-null  int64  
 7   Disposals                89470 non-null  int64  
 8   Kicks                    89470 non-null  int64  
 9   Marks                    89470 non-null  int64  
 10  Handballs                89470 non-null  int64  
 11  Goals                    89470 non-null  int64  
 12  Behinds                  89470 non-null  int64  
 13  Hit Outs                 89470 non-null  int64  
 14  Tackles               

### Storing data for future use before manipulating dataframe

In [7]:
year_list = list(df['year_x'].unique())

## Cleaning Data:
### Fixing scientific notition of rainfall data

In [8]:
df['rainfall'] = df['rainfall'].apply(lambda x: '%.3f' % x)

### Converting date and time objects to datetime:

In [9]:
df = df[['playerId', 'displayName_x', 'height', 'weight', 'dob', 'position', 
        'gameId', 'team', 'year_x', 'round_x', 'Disposals', 'Kicks', 'Marks', 
        'Handballs', 'Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 
        'Inside 50s', 'Clearances', 'Clangers', 'Frees', 'Frees Against',
        'Brownlow Votes', 'Contested Possessions', 'Uncontested Possessions', 
        'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces',
        'Goal Assists', '% Played', 'Subs', 'gameNumber',  'date', 'venue',
        'startTime', 'attendance', 'homeTeam', 'homeTeamScore', 'awayTeam', 
        'awayTeamScore', 'rainfall']]

df.rename({'year_x': 'year', 'round_x': 'round', 'displayName_x': 'displayName'},
          axis=1, inplace=True)

In [10]:
date_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
             'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [11]:
dob = df['dob'].str.split('-', expand=True)
dob[1] = df['dob'].str.split('-', expand=True)[1].replace(date_dict)
df['dob'] = dob.apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df['dob']

0        12-02-2002
1        22-09-1992
2        31-12-1999
3         1-03-1997
4        20-11-1996
            ...    
89465    27-06-1989
89466    11-05-1993
89467    11-02-1993
89468    24-10-1992
89469     4-09-1989
Name: dob, Length: 89470, dtype: object

In [12]:
date = df['date'].str.split('-', expand=True)
date[1] = df['date'].str.split('-', expand=True)[1].replace(date_dict)
df['date'] = date.apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df['date']

0        20-03-2021
1        20-03-2021
2        20-03-2021
3        20-03-2021
4        20-03-2021
            ...    
89465    02-09-2012
89466    02-09-2012
89467    02-09-2012
89468    02-09-2012
89469    02-09-2012
Name: date, Length: 89470, dtype: object

In [13]:
df['dob'] = pd.to_datetime(df['dob'], format='%d'+'-'+'%m'+'-'+'%Y')
df['date'] = pd.to_datetime(df['date'], format='%d'+'-'+'%m'+'-'+'%Y')

In [14]:
df['startTime'] = pd.to_datetime(df['startTime'], format= '%I:%M %p').dt.time
df['startTime']

0        16:05:00
1        16:05:00
2        16:05:00
3        16:05:00
4        16:05:00
           ...   
89465    16:40:00
89466    16:40:00
89467    16:40:00
89468    16:40:00
89469    16:40:00
Name: startTime, Length: 89470, dtype: object

### Fixing 'round' column by making Finals rounds numerical

In [15]:
round_list = {'QF': '24', 'SF': '25', 'PF': '26', 'GF': '27', 'EF': '24'}
df['round'] = df['round'].replace(round_list)
df['round'] = df['round'].str.replace('R', '').astype(float)

### Constructing index

In [16]:
df = df.set_index(['year','round'], append=True)
df = df.sort_index(level=['year', 'round'])

## Feature Creation:
### Age

In [17]:
df.insert(2, 'age', (df['date'] - df['dob']) / np.timedelta64(1,'Y'))

df.drop('dob', axis=1, inplace=True)

### Total Points Scored 

In [18]:
df['Total Points Scored'] = df['Goals'] * 6 + df['Behinds']
df['Total Points Scored']

       year  round
80362  2012  1.00     17
80363  2012  1.00     12
80364  2012  1.00      0
80365  2012  1.00      9
80366  2012  1.00      0
                      ..
9517   2021  27.00    18
9518   2021  27.00     0
9519   2021  27.00     0
9520   2021  27.00     0
9521   2021  27.00     0
Name: Total Points Scored, Length: 89470, dtype: int64

### Is Home Team

In [19]:
df['Is Home Team'] = np.where(df['team'] == df['homeTeam'], 1, 0)
df['Is Home Team']

       year  round
80362  2012  1.00     0
80363  2012  1.00     0
80364  2012  1.00     0
80365  2012  1.00     0
80366  2012  1.00     0
                     ..
9517   2021  27.00    0
9518   2021  27.00    0
9519   2021  27.00    0
9520   2021  27.00    0
9521   2021  27.00    0
Name: Is Home Team, Length: 89470, dtype: int32

### Score Margin (from the perspective of each player)

In [20]:
df['Margin'] = np.where(df['Is Home Team'] == 1,
                        df['homeTeamScore'] - df['awayTeamScore'],
                        df['awayTeamScore'] - df['homeTeamScore'])
df['Margin']

       year  round
80362  2012  1.00     69
80363  2012  1.00     69
80364  2012  1.00     69
80365  2012  1.00     69
80366  2012  1.00     69
                      ..
9517   2021  27.00   -74
9518   2021  27.00   -74
9519   2021  27.00   -74
9520   2021  27.00   -74
9521   2021  27.00   -74
Name: Margin, Length: 89470, dtype: int64

### Game Result

In [21]:
conditions = [df['Margin'] > 0, df['Margin'] == 0, df['Margin'] < 0]
choices = [1, .5, 0]
df['Game Result'] = np.select(conditions, choices)
df['Game Result']

       year  round
80362  2012  1.00    1.00
80363  2012  1.00    1.00
80364  2012  1.00    1.00
80365  2012  1.00    1.00
80366  2012  1.00    1.00
                     ... 
9517   2021  27.00   0.00
9518   2021  27.00   0.00
9519   2021  27.00   0.00
9520   2021  27.00   0.00
9521   2021  27.00   0.00
Name: Game Result, Length: 89470, dtype: float64

### Shifting (offsetting) and averaging certain features by AFL round

In [22]:
shift_columns = ['Disposals', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 
                'Hit Outs', 'Tackles', 'Rebounds', 'Inside 50s', 'Clearances', 
                'Clangers', 'Frees', 'Frees Against', 'Brownlow Votes', 
                'Contested Possessions', 'Uncontested Possessions', 'Contested Marks',
                'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists',
                '% Played', 'Subs', 'venue', 'homeTeamScore', 'awayTeamScore', 
                'rainfall', 'Margin']

average_and_sum_columns = [col for col in shift_columns if not in ['venue', 'Margin', 'rainfall', 'Subs']]

In [23]:
def offset_function(dataframe, column_list, year_list):
    for col in column_list:
        for i in range(1,6): # Shifting 1 to 5
            for n, year in enumerate(year_list):
                year_df = dataframe.xs(year, level=1, drop_level=False)
                year_df = year_df.sort_index('round')
                df_shift = year_df.groupby(['playerId'])[col].shift(-1*i)
                if n != 0:
                    df = pd.concat([df, df_shift])
                else:
                    df = df_shift
            dataframe = pd.concat((dataframe, df.rename(col+f'_shift_{i}')), axis=1)
    return dataframe

In [24]:
df = offset_function(df, shift_columns, year_list)

In [ ]:
def running_average_function(dataframe):
    pass

In [ ]:
def current_year_running_sums(dataframe):
    pass

In [ ]:
def previous_year_running_sums(dataframe):
    pass

In [ ]:
df.xs(2019,level=1).groupby('playerId')['Disposals'].shift(-1)

In [ ]:
df.xs(2020,level=1)['round'].value_counts()

In [ ]:
df.groupby(['playerId'])['Disposals'].shift(-1)

In [ ]:
df.isna().sum()

## Creating AFL Fantasy table based on relative point distributions
### AFL Fantasy:

In [ ]:
afl_fantasy_points = {'Kicks': 3,
                    'Marks': 3,
                    'Handballs': 2,
                    'Goals': 6,
                    'Behinds': 1,
                    'Hit Outs': 1,
                    'Tackles': 4,
                    'Frees': 1,
                    'Frees Against': -3}

In [ ]:
from functools import reduce

fantasy_list = []
for col_name, pts in afl_fantasy_points.items():
    col_vals = df[col_name] * pts
    df[col_name + '_fantasy'] = col_vals
    fantasy_list.append(col_vals)
df['Total Fantasy'] = reduce((lambda x, y: x + y), fantasy_list)